In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests, json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np
import tqdm as tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils 


from torchvision import transforms

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

import torch.utils.data as Data
import numpy as np
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [41]:
from MyDataLoader import MyDataLoader
mdl = MyDataLoader('G')
train_loader, val_loader, test_loader = mdl.get_dataloaders()
Nc, Ns, Nt = mdl.Nc, mdl.Ns, mdl.Nt
nM = mdl.nM
N2V = mdl.N2V
print(Nc, Ns, Nt)

gangnam_dataset_loaded
15 212 16


In [49]:
subcat = '''통신판매업	0	Shop&Service
일반음식점	1	Food
즉석판매제조가공업	2	Food
휴게음식점	3	Food
건강기능식품일반판매업	4	Shop&Service
미용업	5	Shop&Service
출판사	6	Professional & Other Places
담배소매업	7	Shop&Service
의료기기판매(임대)업	8	Shop&Service
의원	9	Professional & Other Places
축산판매업	10	Shop&Service
안전상비의약품 판매업소	11	Shop&Service
방문판매업	12	Shop&Service
식품자동판매기업	13	Shop&Service
인터넷컴퓨터게임시설제공업	14	Shop&Service
약국	15	Shop&Service
집단급식소	16	College&University
노래연습장업	17	Shop&Service
유통전문판매업	18	Shop&Service
제과점영업	19	Shop&Service
세탁업	20	Shop&Service
유료직업소개소	21	Professional & Other Places
국외여행업	22	Travel&Transport
건물위생관리업	23	Shop&Service
비디오물제작업	24	Shop&Service
당구장업	25	Shop&Service
옥외광고업	26	Professional & Other Places
게임물제작업	27	Professional & Other Places
일반여행업	28	Travel&Transport
영화제작업	29	Professional & Other Places
식품제조가공업	30	Shop&Service
이용업	31	Shop&Service
전화권유판매업	32	Shop&Service
식품소분업	33	Shop&Service
숙박업	34	Travel&Transport
인쇄사	35	Shop&Service
행정사업	36	Professional & Other Places
동물판매업	37	Shop&Service
체력단련장업	38	Outdoors & Recreation
대중문화예술기획업	39	Professional & Other Places
건강기능식품유통전문판매업	40	Shop&Service
민방위대피시설	41	Professional & Other Places
체육도장업	42	Outdoors & Recreation
대기오염물질배출시설설치사업장	43	Professional & Other Places
단란주점영업	44	Nightlife Spot
안경업	45	Shop&Service
유흥주점영업	46	Nightlife Spot
국내여행업	47	Travel&Transport
게임물배급업	48	Professional & Other Places
쓰레기종량제봉투판매업	49	Professional & Other Places
골프연습장업	50	Outdoors & Recreation
위탁급식영업	51	College&University
청소년게임제공업	52	Shop&Service
외국인관광도시민박업	53	Travel&Transport
동물약국	54	Shop&Service
소독업	55	Professional & Other Places
고압가스업	56	Professional & Other Places
복합유통게임제공업	57	Professional & Other Places
음반.음악영상물제작업	58	Professional & Other Places
일반게임제공업	59	Shop&Service
사료제조업	60	Professional & Other Places
집단급식소식품판매업	61	Professional & Other Places
치과기공소	62	Professional & Other Places
석유판매업	63	Professional & Other Places
영화배급업	64	Professional & Other Places
목욕장업	65	Shop&Service
동물병원	66	Shop&Service
민방위급수시설	67	Professional & Other Places
식육포장처리업	68	Professional & Other Places
영화수입업	69	Professional & Other Places
저수조청소업	70	Professional & Other Places
식품판매업(기타)	71	Professional & Other Places
공연장	72	Outdoors & Recreation
비디오물배급업	73	Professional & Other Places
의료기기수리업	74	Professional & Other Places
다단계판매업체	75	Professional & Other Places
대규모점포	76	Professional & Other Places
축산가공업	77	Professional & Other Places
국제회의기획업	78	Professional & Other Places
후원방문판매업체	79	Professional & Other Places
영화상영관	80	Outdoors & Recreation
병원	81	Shop&Service
관광사업자	82	Travel&Transport
관광숙박업	83	Travel&Transport
특정고압가스업	84	Professional & Other Places
식품운반업	85	Professional & Other Places
무료직업소개소	86	Professional & Other Places
비디오물시청제공업	87	Professional & Other Places
유원시설업(기타)	88	Outdoors & Recreation
음반.음악영상물배급업	89	Professional & Other Places
건설폐기물처리업	90	Professional & Other Places
환경전문공사업	91	Professional & Other Places
목재수입유통업	92	Professional & Other Places
담배도매업	93	Professional & Other Places
단독정화조/오수처리시설설계시공업	94	Professional & Other Places
의료유사업	95	Professional & Other Places
축산물운반업	96	Professional & Other Places
비디오물감상실업	97	Shop&Service
지하수시공업체	98	Professional & Other Places
영화상영업	99	Shop&Service
승강기유지관리업체	100	Professional & Other Places
동물용의료용구판매업	101	Professional & Other Places
담배수입판매업체	102	Professional & Other Places
석유및석유대체연료판매업체	103	Professional & Other Places
한옥체험업	104	Travel&Transport
산후조리업	105	Shop&Service
수영장업	106	Outdoors & Recreation
관광식당	107	Travel&Transport
환경측정대행업	108	Professional & Other Places
요양보호사교육기관	109	Shop&Service
무도학원업	110	Shop&Service
종합체육시설업	111	Outdoors & Recreation
용기·포장지제조업	112	Professional & Other Places
지하수영향조사기관	113	Professional & Other Places
지하수정화업체	114	Professional & Other Places
상조업	115	Shop&Service
분뇨수집운반업	116	Professional & Other Places'''

In [65]:
subdict = dict()
agdict = dict()
for sent in subcat.split('\n'):
    a, b, c = sent.split('\t')
    if c not in subdict:
        subdict[c] = len(subdict)
        
    agdict.setdefault(subdict[c], [])
    agdict[subdict[c]].append(int(b))

In [78]:
import json

with open('subdict.json', 'w') as fp:
    json.dump(agdict, fp)

In [79]:

with open('subdict.json') as fp:
    sd = json.load(fp)

In [80]:
sd

{'0': [0,
  4,
  5,
  7,
  8,
  10,
  11,
  12,
  13,
  14,
  15,
  17,
  18,
  19,
  20,
  23,
  24,
  25,
  30,
  31,
  32,
  33,
  35,
  37,
  40,
  45,
  52,
  54,
  59,
  65,
  66,
  81,
  97,
  99,
  105,
  109,
  110,
  115],
 '1': [1, 2, 3],
 '2': [6,
  9,
  21,
  26,
  27,
  29,
  36,
  39,
  41,
  43,
  48,
  49,
  55,
  56,
  57,
  58,
  60,
  61,
  62,
  63,
  64,
  67,
  68,
  69,
  70,
  71,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  84,
  85,
  86,
  87,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  98,
  100,
  101,
  102,
  103,
  108,
  112,
  113,
  114,
  116],
 '3': [16, 51],
 '4': [22, 28, 34, 47, 53, 82, 83, 104, 107],
 '5': [38, 42, 50, 72, 80, 88, 106, 111],
 '6': [44, 46]}

In [46]:
mdl.nM.shape

(1769, 212, 117)

In [182]:
Ac = mdl.Ac
As = mdl.As
cmask = Ac #As@As@Ac + As@Ac + Ac
cmask[cmask != 0] = 1
smask = (mdl.DAs + mdl.DAs.T + np.eye(Ns))

In [77]:
np.array([[1, 2, 3], [4, 5, 6]])[[[0, 1], [0, 1]], :]

array([[[1, 2, 3],
        [4, 5, 6]],

       [[1, 2, 3],
        [4, 5, 6]]])

In [197]:
nAc = mdl.Ac / mdl.Ac.sum(axis=0)
c2v = nAc.T @ mdl.N2V

In [201]:
nAc.sum(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [202]:
c2v.shape

(15, 64)

In [204]:
t =torch.tensor([[1, 2], [3, 4]])

In [205]:
t

tensor([[1, 2],
        [3, 4]])

In [211]:
t.repeat(2, 2)

tensor([[1, 2, 1, 2],
        [3, 4, 3, 4],
        [1, 2, 1, 2],
        [3, 4, 3, 4]])

In [234]:
torch.eye(2).repeat_interleave(2, dim=0).repeat_interleave(2, dim=1)

tensor([[1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [0., 0., 1., 1.],
        [0., 0., 1., 1.]])

In [235]:
torch.eye(3).repeat_interleave(4, dim=0).repeat_interleave(2, dim=1)

tensor([[1., 1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0.],
        [0., 0., 1., 1., 0., 0.],
        [0., 0., 1., 1., 0., 0.],
        [0., 0., 1., 1., 0., 0.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1., 1.]])

In [242]:
t[:, 0]

tensor([1, 3])

In [177]:
cmask.T

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [175]:
(cmask.T @ N2V)

(15, 64)

(212, 64)

In [167]:
cmask

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [165]:
mdl.Ac

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [164]:
mdl.N2V

array([[-0.20851435,  0.33834934,  0.3882506 , ..., -0.01472863,
        -0.4250449 ,  0.34857735],
       [-0.05378522,  0.46486378,  0.38714692, ...,  0.1443695 ,
        -0.34835047,  0.40649125],
       [-0.16263969,  0.34185192,  0.17111267, ..., -0.03173962,
        -0.4231226 ,  0.36039332],
       ...,
       [-0.11960851,  0.23162398, -0.26321155, ..., -0.2122293 ,
        -0.55343986, -0.3502737 ],
       [-0.32946458, -0.06882444, -0.3833462 , ..., -0.28586787,
        -0.3687419 , -0.6261262 ],
       [-0.06848181,  0.05162987, -0.5669245 , ..., -0.02540488,
        -0.27357998, -0.60659003]])

In [124]:
import os
perf_dict = dict()
date_dict = dict()
for fname in os.listdir('trainlog'):
    if fname[-4:] == '.txt':
        df = pd.read_csv('trainlog/'+fname, sep='\t')
        name, ymd, hms = fname.split('.')[0].split('_')
        
        if name not in perf_dict:
            perf_dict[name] = [100, 100, 100, 100]
        
        if perf_dict[name][2] > df[df.columns[4]].min():
            perf_dict[name] = [
                df.iloc[df[df.columns[4]].argmin()][2],
                df.iloc[df[df.columns[4]].argmin()][3],
                df.iloc[df[df.columns[4]].argmin()][4],
                df.iloc[df[df.columns[4]].argmin()][5]]
            date_dict[name] = ymd+'_'+hms

for fname in perf_dict:
    print((fname+'____________')[:14], '\t'.join([str(s) for s in perf_dict[fname]]))

In [120]:
df[df.columns[4]].min()

2.4985

In [109]:
df[df.columns[4]].argmin()

58

In [118]:
df.iloc[df[df.columns[4]].argmin()][2]

11.2765

In [141]:
for fname in perf_dict:
    print((fname+'____________')[:14], '\t'.join([str(s) for s in perf_dict[fname]]))

FNNshufC______ 8.8738	2.9789	2.0512	10.865
AttnSTshufC___ 8.6371	2.9389	2.0059	10.6955
FNNattnC______ 11.0971	3.3312	2.356	12.5906
FNN___________ 10.9935	3.3156	2.3756	12.6655


In [107]:
perf_dict

{'FNNshufC': 2.0512, 'AttnSTshufC': 2.0059, 'FNNattnC': 2.356, 'FNN': 2.3756}

In [94]:
df = pd.read_csv('trainlog/'+fname, sep='\t')

2.4655

In [89]:
fname.split('.')[0].split('_')

['FNNattnC', '20210326', '165650']

In [76]:
x = np.hstack((mdl.K, mdl.P[:, np.newaxis]))

In [77]:
x.shape

(1769, 8)

In [83]:
for x, m, c, pk in train_loader:
    break

In [64]:
mdl.P

array([1, 0, 0, ..., 0, 0, 0])

In [ ]:
mdl.

In [58]:
mdl.tr_K

tensor([3, 4, 5,  ..., 5, 6, 0])

In [63]:
torch.eye(7)[mdl.tr_K]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [1., 0., 0.,  ..., 0., 0., 0.]])

In [59]:
mdl.tr_P

tensor([1, 0, 0,  ..., 0, 0, 0])

In [57]:
p

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [36]:
w

tensor([6, 1, 4, 5, 2, 0, 2, 2, 2, 2, 0, 2, 6, 4, 2, 6, 1, 6, 4, 1, 1, 6, 4, 4,
        2, 1, 0, 0, 3, 1, 3, 6, 2, 0, 0, 3, 6, 0, 4, 1, 1, 1, 1, 3, 4, 1, 2, 4])

In [35]:
p

tensor([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
import datetime

In [12]:
dt.astype(datetime.datetime).isoweekday()

4